# 晶体结构生成推理 Notebook - 迭代优化版

本notebook实现迭代优化的推理流程：

## 核心流程
1. **初始推理**：模型推理生成初始submission.csv
2. **质量评估**：使用RWP指标评估PXRD匹配质量
3. **迭代优化循环**：
   - 质量不好的样本进行后处理（能量优化→Rietveld精修）
   - 如果质量改善则更新submission.csv
   - 仍不满足要求的批量重新生成
4. **终止条件**：
   - 总运行时间超过5小时
   - 单个样本尝试次数超限
   - 所有样本满足质量要求

**注意**: 这个notebook设计为可直接在比赛环境运行

## 1. 导入必要的库和设置

## CFG (Classifier-Free Guidance) 使用说明

本notebook默认使用 **cfm_cfg** 流模型进行推理，它支持动态调节生成质量：

### 核心参数
- **CFG_GUIDANCE_SCALE** (默认1.5): 控制条件引导强度
  - `1.0`: 标准条件生成（无增强）
  - `>1.0`: 增强条件控制（更精确匹配PXRD，但可能过拟合）
  - `<1.0`: 增加多样性（更多探索，但可能偏离目标）

### 自适应策略
- **迭代早期** (1-2轮): 标准引导强度，平衡探索
- **迭代中期** (3-5轮): 增强引导强度，精确匹配
- **迭代后期** (>5轮): 降低引导强度，增加多样性
- **困难样本**: 根据原子数量自动调节引导强度

### 使用示例
```python
# 手动调节单个样本的引导强度
structure = model.flow.sample(conditions, guidance_scale=2.0)  # 强引导

# 批量生成时使用不同策略
structures, scales = generate_crystal_structures_batch_cfg(
    df, model, normalizer,
    guidance_scale=1.5,      # 固定引导强度
    adaptive_mode=True       # 或使用自适应模式
)
```

In [14]:
import os
import sys
import time
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import torch
import multiprocessing as mp
import warnings
warnings.filterwarnings('ignore')

# 添加src目录到路径
sys.path.append('.')  # 添加根目录
sys.path.append('src')

# 导入推理工具函数
from inference_utils import (
    # 数据加载
    load_competition_data,
    load_lattice_stats,
    # 模型和推理
    load_model,
    initialize_normalizer,  # 新增的归一化器初始化函数
    generate_crystal_structures_batch,
    generate_crystal_structures_batch_cfg,
    generate_random_structure,
    # PXRD评估
    evaluate_structures_batch,
    evaluate_structure_quality,
    # 流水线处理（新增）
    generate_and_evaluate_batch_simple,
    generate_and_evaluate_pipeline,
    # 迭代控制
    check_termination_conditions,
    get_samples_to_regenerate,
    get_adaptive_cfg_scale,
    # 文件I/O
    update_submission_incrementally,
    log_submission_update,
    print_final_statistics
)

# 导入必要的模块
from src.pxrd_simulator import PXRDSimulator

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA设备: {torch.cuda.get_device_name(0)}")

PyTorch版本: 2.8.0+cu128
CUDA可用: True
CUDA设备: NVIDIA A100-SXM4-80GB


In [15]:
import os
import sys
import time
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import torch
import multiprocessing as mp
import warnings
warnings.filterwarnings('ignore')

# 添加src目录到路径
sys.path.append('.')  # 添加根目录
sys.path.append('src')

# 导入推理工具函数
from inference_utils import (
    # 数据加载
    load_competition_data,
    load_lattice_stats,
    # 模型和推理
    load_model,
    initialize_normalizer,  # 新增的归一化器初始化函数
    generate_crystal_structures_batch,
    generate_crystal_structures_batch_cfg,
    generate_random_structure,
    # PXRD评估
    evaluate_structures_batch,
    # 迭代控制
    check_termination_conditions,
    get_samples_to_regenerate,
    get_adaptive_cfg_scale,
    # 文件I/O
    update_submission_incrementally,
    log_submission_update,
    print_final_statistics
)

# 导入必要的模块
from src.pxrd_simulator import PXRDSimulator

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA设备: {torch.cuda.get_device_name(0)}")

PyTorch版本: 2.8.0+cu128
CUDA可用: True
CUDA设备: NVIDIA A100-SXM4-80GB


## 2. 配置参数

In [16]:
# 数据路径配置
DATA_DIR = Path("data/A_sample")  # 比赛数据目录
COMPOSITION_FILE = DATA_DIR / "composition.json"
PATTERN_DIR = DATA_DIR / "pattern"

# 模型路径 - 使用实际的checkpoint路径
MODEL_PATH = "/home/ma-user/work/mincycle4csp/outputs/crystal_transformer_cfm_cfg_20250829_023539/checkpoints/epoch=epoch=044-val_loss=val/loss=2.6581.ckpt"

# 归一化统计数据路径
LATTICE_STATS_PATH = "data/lattice_stats.json"

# 设备配置
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {DEVICE}")
if DEVICE.type == 'cuda':
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  显存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# 输出文件（必须在根目录）
SUBMISSION_FILE = "submission.csv"

# CFG推理参数
CFG_GUIDANCE_SCALE = 1.5  # 默认CFG引导强度（1.0=标准，>1增强条件控制）
CFG_ADAPTIVE_MODE = True  # 是否使用自适应引导强度
CFG_MIN_SCALE = 0.8  # 自适应模式下的最小引导强度
CFG_MAX_SCALE = 2.5  # 自适应模式下的最大引导强度

# 优化参数
RWP_THRESHOLD = 0.15  # RWP质量阈值，低于此值认为质量合格
MAX_TIME_HOURS = 5  # 最大运行时间（小时）
MAX_ATTEMPTS_PER_SAMPLE = 10  # 每个样本最大尝试次数
BATCH_SIZE = 32  # 批量重新生成的大小

# 记录开始时间
START_TIME = time.time()
MAX_RUNTIME = MAX_TIME_HOURS * 3600  # 转换为秒

print(f"\n配置参数：")
print(f"  模型路径: {MODEL_PATH}")
print(f"  模型存在: {os.path.exists(MODEL_PATH)}")
print(f"  归一化数据: {LATTICE_STATS_PATH}")
print(f"  流模型: cfm_cfg (Classifier-Free Guidance)")
print(f"  CFG引导强度: {CFG_GUIDANCE_SCALE}")
print(f"  自适应模式: {CFG_ADAPTIVE_MODE}")
if CFG_ADAPTIVE_MODE:
    print(f"    引导强度范围: [{CFG_MIN_SCALE}, {CFG_MAX_SCALE}]")
print(f"  RWP阈值: {RWP_THRESHOLD}")
print(f"  最大运行时间: {MAX_TIME_HOURS}小时")
print(f"  单样本最大尝试: {MAX_ATTEMPTS_PER_SAMPLE}次")
print(f"  开始时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

使用设备: cuda
  GPU: NVIDIA A100-SXM4-80GB
  显存: 79.3 GB

配置参数：
  模型路径: /home/ma-user/work/mincycle4csp/outputs/crystal_transformer_cfm_cfg_20250829_023539/checkpoints/epoch=epoch=044-val_loss=val/loss=2.6581.ckpt
  模型存在: True
  归一化数据: data/lattice_stats.json
  流模型: cfm_cfg (Classifier-Free Guidance)
  CFG引导强度: 1.5
  自适应模式: True
    引导强度范围: [0.8, 2.5]
  RWP阈值: 0.15
  最大运行时间: 5小时
  单样本最大尝试: 10次
  开始时间: 2025-08-29 12:06:14


## 3. 数据加载函数

In [17]:
# 数据加载函数已移至inference_utils.py
# 这里直接使用导入的函数

## 4. 加载数据

In [18]:
# 加载比赛数据
df = load_competition_data(DATA_DIR)
print(f"\n加载了 {len(df)} 个样本")
print(f"数据列: {df.columns.tolist()}")
print(f"\n前5个样本:")
print(df[['id', 'niggli_comp', 'num_atoms']].head())

# 加载归一化统计信息
lattice_stats = load_lattice_stats(LATTICE_STATS_PATH)

# 初始化样本状态追踪
sample_status = {
    sample_id: {
        'attempts': 0,
        'best_rwp': float('inf'),
        'best_structure': None,
        'satisfied': False
    }
    for sample_id in df['id']
}

print(f"\n初始化了 {len(sample_status)} 个样本的状态追踪")

加载数据:   0%|          | 0/200 [00:00<?, ?it/s]


加载了 200 个样本
数据列: ['id', 'niggli_comp', 'primitive_comp', 'atom_types', 'num_atoms', 'pxrd']

前5个样本:
       id         niggli_comp  num_atoms
0   A-329        Sb3 Sc10 Te7         20
1  A-1447       B3 Mg1 N6 Sr4         14
2  A-1150  Ba1 Nd1 O6 Os1 Sr1         10
3   A-559         Eu1 Ga3 Zn1          5
4  A-1956      Ba4 Gd1 Nb1 O8         14
✅ 加载归一化统计信息: data/lattice_stats.json
   晶格均值范围: [-0.48, 2.02]
   晶格标准差范围: [1.85, 6.26]

初始化了 200 个样本的状态追踪


## 5. 模型和推理函数

In [19]:
# 模型加载和推理函数已移至inference_utils.py
# 这里只保留模型初始化代码

def initialize_model_and_tools(model_path, lattice_stats_path="data/lattice_stats.json"):
    """
    初始化模型和工具
    
    Args:
        model_path: checkpoint文件路径
        lattice_stats_path: 晶格统计信息文件路径
        
    Returns:
        tuple: (模型, 归一化器, PXRD仿真器)
    """
    try:
        # 加载模型
        model = load_model(model_path)
        
        # 初始化数据归一化器（使用新的初始化函数）
        data_normalizer = initialize_normalizer(lattice_stats_path)
        
        # 初始化PXRD仿真器
        pxrd_simulator = PXRDSimulator()
        
        print("✅ 模型和工具初始化成功")
        return model, data_normalizer, pxrd_simulator
        
    except Exception as e:
        print(f"⚠️ 模型加载失败: {e}")
        print("将使用随机生成作为备用方案")
        return None, None, None

# 加载模型和初始化工具
model, data_normalizer, pxrd_simulator = initialize_model_and_tools(MODEL_PATH, LATTICE_STATS_PATH)

正在加载模型: /home/ma-user/work/mincycle4csp/outputs/crystal_transformer_cfm_cfg_20250829_023539/checkpoints/epoch=epoch=044-val_loss=val/loss=2.6581.ckpt
  检测到流模型: cfm_cfg
✅ 模型加载成功，设备: cuda
✅ 使用文件初始化归一化器: data/lattice_stats.json
✅ 模型和工具初始化成功


## 6. PXRD计算和质量评估

In [20]:
# PXRD计算和质量评估函数已移至inference_utils.py
# 这里直接使用导入的函数

## 7. 后处理函数

In [21]:
# 后处理函数已移至inference_utils.py
# 这里直接使用导入的函数

## 8. 终止条件检查

In [22]:
# 终止条件检查函数已移至inference_utils.py
# 这里直接使用导入的函数

print("="*60)
print("阶段1：初始推理（流水线批处理）")
print("="*60)

# 配置批处理参数
INFERENCE_BATCH_SIZE = 32  # GPU推理批大小
PXRD_WORKERS = min(mp.cpu_count() // 2, 8)  # PXRD计算并行进程数
PXRD_TIMEOUT = 120  # PXRD计算超时时间（秒）

print(f"批处理配置:")
print(f"  推理批大小: {INFERENCE_BATCH_SIZE}")
print(f"  PXRD并行进程: {PXRD_WORKERS}")
print(f"  PXRD超时: {PXRD_TIMEOUT}秒")

initial_predictions = {}

if model is not None:
    # 使用流水线批量生成和评测
    print(f"\n使用流水线处理 {len(df)} 个结构...")
    
    # 使用简化版流水线（生成一批立即评测）
    all_structures, all_rwp_values = generate_and_evaluate_batch_simple(
        df, 
        model, 
        data_normalizer, 
        batch_size=INFERENCE_BATCH_SIZE,
        n_workers=PXRD_WORKERS,
        timeout_seconds=PXRD_TIMEOUT
    )
    
    # 更新样本状态
    for sample_id, structure, rwp_value in zip(df['id'], all_structures, all_rwp_values):
        sample_status[sample_id]['attempts'] = 1
        sample_status[sample_id]['best_rwp'] = rwp_value
        sample_status[sample_id]['best_structure'] = structure
        sample_status[sample_id]['satisfied'] = rwp_value < RWP_THRESHOLD
        initial_predictions[sample_id] = structure
        
else:
    # 备用：逐个生成随机结构
    print("⚠️ 使用随机生成（模型未加载）...")
    from tqdm.auto import tqdm
    from inference_utils import evaluate_structure_quality
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="生成随机结构"):
        sample_id = row['id']
        structure = generate_random_structure(row.to_dict())
        rwp_value = evaluate_structure_quality(structure, row['pxrd'], pxrd_simulator)
        
        sample_status[sample_id]['attempts'] = 1
        sample_status[sample_id]['best_rwp'] = rwp_value
        sample_status[sample_id]['best_structure'] = structure
        sample_status[sample_id]['satisfied'] = rwp_value < RWP_THRESHOLD
        initial_predictions[sample_id] = structure

# 统计初始结果
satisfied_count = sum(1 for s in sample_status.values() if s['satisfied'])
avg_rwp = np.mean([s['best_rwp'] for s in sample_status.values()])

print(f"\n初始推理结果:")
print(f"  总样本数: {len(sample_status)}")
print(f"  满足要求: {satisfied_count}/{len(sample_status)} ({satisfied_count/len(sample_status)*100:.1f}%)")
print(f"  平均RWP: {avg_rwp:.4f}")
print(f"  RWP阈值: {RWP_THRESHOLD}")

# 显示RWP分布
rwp_values = [s['best_rwp'] for s in sample_status.values()]
print(f"\nRWP分布:")
print(f"  最小值: {np.min(rwp_values):.4f}")
print(f"  25%分位: {np.percentile(rwp_values, 25):.4f}")
print(f"  中位数: {np.median(rwp_values):.4f}")
print(f"  75%分位: {np.percentile(rwp_values, 75):.4f}")
print(f"  最大值: {np.max(rwp_values):.4f}")

# 立即保存初始推理结果到submission.csv
submission_df = update_submission_incrementally(sample_status, DATA_DIR, SUBMISSION_FILE)
log_submission_update(0, sample_status, SUBMISSION_FILE)

In [23]:
print("="*60)
print("阶段1：初始推理（批量处理）")
print("="*60)

# 配置批处理参数
INFERENCE_BATCH_SIZE = 32  # GPU推理批大小
PXRD_WORKERS = min(mp.cpu_count() // 2, 64)  # PXRD计算并行进程数

print(f"批处理配置:")
print(f"  推理批大小: {INFERENCE_BATCH_SIZE}")
print(f"  PXRD并行进程: {PXRD_WORKERS}")

initial_predictions = {}

if model is not None:
    # 使用模型批量生成
    print(f"\n使用模型批量生成 {len(df)} 个结构...")
    
    # 批量生成所有结构
    all_structures = generate_crystal_structures_batch(
        df, 
        model, 
        data_normalizer, 
        batch_size=INFERENCE_BATCH_SIZE
    )
    
    # 批量评估质量
    print("批量计算PXRD和评估质量...")
    all_observed_pxrds = df['pxrd'].tolist()
    all_rwp_values = evaluate_structures_batch(
        all_structures,
        all_observed_pxrds,
        n_workers=PXRD_WORKERS
    )
    
    # 更新样本状态
    for idx, (sample_id, structure, rwp_value) in enumerate(zip(df['id'], all_structures, all_rwp_values)):
        sample_status[sample_id]['attempts'] = 1
        sample_status[sample_id]['best_rwp'] = rwp_value
        sample_status[sample_id]['best_structure'] = structure
        sample_status[sample_id]['satisfied'] = rwp_value < RWP_THRESHOLD
        initial_predictions[sample_id] = structure
        
        # 每处理50个样本显示一次进度
        if (idx + 1) % 50 == 0:
            print(f"  处理进度: {idx + 1}/{len(df)}")
else:
    # 备用：逐个生成随机结构
    print("⚠️ 使用随机生成（模型未加载）...")
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="生成随机结构"):
        sample_id = row['id']
        structure = generate_random_structure(row)
        rwp_value = evaluate_structure_quality(structure, row['pxrd'], pxrd_simulator)
        
        sample_status[sample_id]['attempts'] = 1
        sample_status[sample_id]['best_rwp'] = rwp_value
        sample_status[sample_id]['best_structure'] = structure
        sample_status[sample_id]['satisfied'] = rwp_value < RWP_THRESHOLD
        initial_predictions[sample_id] = structure

# 统计初始结果
satisfied_count = sum(1 for s in sample_status.values() if s['satisfied'])
avg_rwp = np.mean([s['best_rwp'] for s in sample_status.values()])

print(f"\n初始推理结果:")
print(f"  总样本数: {len(sample_status)}")
print(f"  满足要求: {satisfied_count}/{len(sample_status)} ({satisfied_count/len(sample_status)*100:.1f}%)")
print(f"  平均RWP: {avg_rwp:.4f}")
print(f"  RWP阈值: {RWP_THRESHOLD}")

# 显示RWP分布
rwp_values = [s['best_rwp'] for s in sample_status.values()]
print(f"\nRWP分布:")
print(f"  最小值: {np.min(rwp_values):.4f}")
print(f"  25%分位: {np.percentile(rwp_values, 25):.4f}")
print(f"  中位数: {np.median(rwp_values):.4f}")
print(f"  75%分位: {np.percentile(rwp_values, 75):.4f}")
print(f"  最大值: {np.max(rwp_values):.4f}")

# 立即保存初始推理结果到submission.csv
submission_df = update_submission_incrementally(sample_status, DATA_DIR, SUBMISSION_FILE)
log_submission_update(0, sample_status, SUBMISSION_FILE)

阶段1：初始推理（批量处理）
批处理配置:
  推理批大小: 32
  PXRD并行进程: 64

使用模型批量生成 200 个结构...
使用CFG采样: 1.5
使用CFG采样: 1.5
使用CFG采样: 1.5
使用CFG采样: 1.5
使用CFG采样: 1.5
使用CFG采样: 1.5
使用CFG采样: 1.5
批量计算PXRD和评估质量...
开始批量评估 200 个结构...
批量计算 200 个PXRD谱 (并行数=64, 超时=120秒)
  进度: 10/200 (成功=10, 失败=0)
  进度: 20/200 (成功=20, 失败=0)
  进度: 30/200 (成功=30, 失败=0)
  进度: 40/200 (成功=40, 失败=0)
  进度: 50/200 (成功=50, 失败=0)
  进度: 60/200 (成功=60, 失败=0)
  进度: 70/200 (成功=70, 失败=0)
  进度: 80/200 (成功=80, 失败=0)
  进度: 90/200 (成功=90, 失败=0)
  进度: 100/200 (成功=100, 失败=0)
  进度: 110/200 (成功=110, 失败=0)
  进度: 120/200 (成功=120, 失败=0)
  进度: 130/200 (成功=130, 失败=0)
  进度: 140/200 (成功=140, 失败=0)
  进度: 150/200 (成功=150, 失败=0)
  进度: 160/200 (成功=160, 失败=0)
  进度: 170/200 (成功=170, 失败=0)
  进度: 180/200 (成功=180, 失败=0)
  进度: 190/200 (成功=190, 失败=0)
  进度: 200/200 (成功=200, 失败=0)
✅ PXRD计算完成: 全部200个成功
评估完成: 耗时 3.30秒
  处理进度: 50/200
  处理进度: 100/200
  处理进度: 150/200
  处理进度: 200/200

初始推理结果:
  总样本数: 200
  满足要求: 0/200 (0.0%)
  平均RWP: 108.9434
  RWP阈值: 0.15

RWP分布:
  最小值: 5.2217
  25%分位: 71.7

## 13. 迭代优化循环（带实时保存）

In [ ]:
print("\n" + "="*60)
print("阶段2：迭代优化（批量处理 + CFG动态调节）")
print("="*60)

iteration = 0
while True:
    iteration += 1
    
    # 检查终止条件（使用导入的函数）
    should_terminate, reason = check_termination_conditions(
        sample_status, START_TIME, MAX_RUNTIME, MAX_ATTEMPTS_PER_SAMPLE
    )
    if should_terminate:
        print(f"\n终止优化: {reason}")
        break
    
    print(f"\n--- 迭代 {iteration} ---")
    elapsed = time.time() - START_TIME
    print(f"已运行: {elapsed/3600:.2f}小时")
    
    # 获取需要优化的样本（使用导入的函数）
    samples_to_process = get_samples_to_regenerate(
        sample_status, BATCH_SIZE, MAX_ATTEMPTS_PER_SAMPLE
    )
    
    if not samples_to_process:
        print("没有需要处理的样本")
        break
    
    print(f"处理 {len(samples_to_process)} 个样本")
    
    # 准备批处理数据
    batch_df = df[df['id'].isin(samples_to_process)]
    
    # 标记是否有改进
    has_improvement = False
    
    # 获取自适应的CFG引导强度（使用导入的函数）
    current_cfg_scale = get_adaptive_cfg_scale(
        iteration, CFG_GUIDANCE_SCALE, CFG_MIN_SCALE, CFG_MAX_SCALE
    )
    
    if iteration <= 2:
        print(f"  CFG策略：早期探索，引导强度={current_cfg_scale:.2f}")
    elif iteration <= 5:
        print(f"  CFG策略：精确匹配，引导强度={current_cfg_scale:.2f}")
    else:
        print(f"  CFG策略：增加多样性，引导强度={current_cfg_scale:.2f}")
    
    # 对于多次失败的样本，使用更激进的引导策略
    difficult_samples = [sid for sid in samples_to_process 
                        if sample_status[sid]['attempts'] >= 5]
    
    if difficult_samples:
        print(f"  发现 {len(difficult_samples)} 个困难样本，使用自适应CFG")
    
    # 策略2：批量重新生成（使用动态CFG）
    if model is not None:
        print(f"  批量重新生成 {len(batch_df)} 个结构...")
        
        # 为困难样本使用不同的引导强度
        if difficult_samples:
            # 分两批处理：困难样本和普通样本
            difficult_df = batch_df[batch_df['id'].isin(difficult_samples)]
            normal_df = batch_df[~batch_df['id'].isin(difficult_samples)]
            
            new_structures = []
            scales_used = []
            
            # 困难样本：使用自适应CFG
            if len(difficult_df) > 0:
                print(f"    处理困难样本（自适应CFG）...")
                diff_structures, diff_scales = generate_crystal_structures_batch_cfg(
                    difficult_df,
                    model,
                    data_normalizer,
                    batch_size=INFERENCE_BATCH_SIZE,
                    guidance_scale=None,  # 使用自适应
                    adaptive_mode=True,
                    min_scale=CFG_MIN_SCALE,
                    max_scale=CFG_MAX_SCALE
                )
                new_structures.extend(diff_structures)
                scales_used.extend(diff_scales)
                
                # 显示使用的引导强度分布
                print(f"      引导强度范围: [{min(diff_scales):.2f}, {max(diff_scales):.2f}]")
            
            # 普通样本：使用当前迭代的引导强度
            if len(normal_df) > 0:
                print(f"    处理普通样本（CFG={current_cfg_scale:.2f}）...")
                norm_structures, norm_scales = generate_crystal_structures_batch_cfg(
                    normal_df,
                    model,
                    data_normalizer,
                    batch_size=INFERENCE_BATCH_SIZE,
                    guidance_scale=current_cfg_scale,
                    adaptive_mode=False
                )
                new_structures.extend(norm_structures)
                scales_used.extend(norm_scales)
            
            # 重新排序以匹配原始batch_df顺序
            id_to_structure = dict(zip(
                list(difficult_df['id']) + list(normal_df['id']),
                new_structures
            ))
            id_to_scale = dict(zip(
                list(difficult_df['id']) + list(normal_df['id']),
                scales_used
            ))
            
            new_structures = [id_to_structure[sid] for sid in batch_df['id']]
            scales_used = [id_to_scale[sid] for sid in batch_df['id']]
            
        else:
            # 所有样本使用相同的引导强度
            new_structures, scales_used = generate_crystal_structures_batch_cfg(
                batch_df,
                model,
                data_normalizer,
                batch_size=INFERENCE_BATCH_SIZE,
                guidance_scale=current_cfg_scale,
                adaptive_mode=False
            )
        
        # 批量评估新结构
        print("  批量评估新结构质量...")
        batch_observed_pxrds = batch_df['pxrd'].tolist()
        new_rwp_values = evaluate_structures_batch(
            new_structures,
            batch_observed_pxrds,
            n_workers=PXRD_WORKERS
        )
        
        # 更新状态（保留最佳结果）
        improvements = []
        for sample_id, new_structure, new_rwp, used_scale in zip(
            batch_df['id'], new_structures, new_rwp_values, scales_used
        ):
            current_best_rwp = sample_status[sample_id]['best_rwp']
            
            # 如果新结构更好，则更新
            if new_rwp < current_best_rwp:
                improvement_ratio = (current_best_rwp - new_rwp) / current_best_rwp
                improvements.append((sample_id, improvement_ratio, used_scale))
                
                sample_status[sample_id]['best_structure'] = new_structure
                sample_status[sample_id]['best_rwp'] = new_rwp
                sample_status[sample_id]['satisfied'] = new_rwp < RWP_THRESHOLD
                has_improvement = True
            
            # 更新尝试次数
            sample_status[sample_id]['attempts'] += 1
        
        # 显示改进最大的样本
        if improvements:
            improvements.sort(key=lambda x: x[1], reverse=True)
            print(f"\n  最佳改进样本:")
            for sid, ratio, scale in improvements[:3]:  # 显示前3个
                print(f"    {sid}: 改进{ratio*100:.1f}% (CFG={scale:.2f})")
    
    else:
        # 备用：逐个处理（使用随机生成）
        from tqdm.auto import tqdm
        for sample_id in tqdm(samples_to_process, desc=f"迭代{iteration}"):
            row = df[df['id'] == sample_id].iloc[0]
            new_structure = generate_random_structure(row.to_dict())
            
            # 使用导入的评估函数
            from inference_utils import evaluate_structure_quality
            new_rwp = evaluate_structure_quality(new_structure, row['pxrd'], pxrd_simulator)
            
            current_best_rwp = sample_status[sample_id]['best_rwp']
            if new_rwp < current_best_rwp:
                sample_status[sample_id]['best_structure'] = new_structure
                sample_status[sample_id]['best_rwp'] = new_rwp
                sample_status[sample_id]['satisfied'] = new_rwp < RWP_THRESHOLD
                has_improvement = True
            
            sample_status[sample_id]['attempts'] += 1
    
    # 统计当前状态
    satisfied_count = sum(1 for s in sample_status.values() if s['satisfied'])
    avg_rwp = np.mean([s['best_rwp'] for s in sample_status.values()])
    
    print(f"\n迭代{iteration}结果:")
    print(f"  满足要求: {satisfied_count}/{len(sample_status)} ({satisfied_count/len(sample_status)*100:.1f}%)")
    print(f"  平均RWP: {avg_rwp:.4f}")
    
    if has_improvement:
        print("  ✨ 本轮有样本得到改进")
        
        # 显示改进的样本数
        improved_count = sum(1 for sid in samples_to_process 
                            if sample_status[sid]['attempts'] > 1 
                            and sample_status[sid]['satisfied'])
        if improved_count > 0:
            print(f"  📈 新满足要求的样本: {improved_count}")
    
    # 每次迭代后都更新submission.csv（使用导入的函数）
    submission_df = update_submission_incrementally(sample_status, DATA_DIR, SUBMISSION_FILE)
    log_submission_update(iteration, sample_status, SUBMISSION_FILE)
    
    # 限制迭代次数（额外保护）
    if iteration > 100:
        print("达到最大迭代次数")
        break
    
    # 提前终止：如果没有改进且尝试次数较多
    if not has_improvement and iteration > 5:
        print("连续多轮无改进，提前终止")
        break

print("\n" + "="*60)
print("迭代优化完成")
print("="*60)


阶段2：迭代优化（批量处理 + CFG动态调节）

--- 迭代 1 ---
已运行: 0.01小时
处理 32 个样本
  CFG策略：早期探索，引导强度=1.50
  批量重新生成 32 个结构...
使用CFG采样: 1.5


  批量评估新结构质量...
开始批量评估 32 个结构...
批量计算 32 个PXRD谱 (并行数=64, 超时=120秒)
  进度: 10/32 (成功=10, 失败=0)
  进度: 20/32 (成功=20, 失败=0)
  进度: 30/32 (成功=30, 失败=0)
✅ PXRD计算完成: 全部32个成功
评估完成: 耗时 2.28秒

  最佳改进样本:
    A-1842: 改进81.5% (CFG=1.50)
    A-1925: 改进67.6% (CFG=1.50)
    A-1956: 改进51.8% (CFG=1.50)

迭代1结果:
  满足要求: 0/200 (0.0%)
  平均RWP: 101.2661
  ✨ 本轮有样本得到改进

📝 submission.csv已更新 (迭代1)
   满足要求: 0/200 (0.0%)
   文件大小: 274.63 KB

--- 迭代 2 ---
已运行: 0.01小时
处理 32 个样本
  CFG策略：早期探索，引导强度=1.50
  批量重新生成 32 个结构...
使用CFG采样: 1.5
  批量评估新结构质量...
开始批量评估 32 个结构...
批量计算 32 个PXRD谱 (并行数=64, 超时=120秒)
  进度: 10/32 (成功=10, 失败=0)
  进度: 20/32 (成功=20, 失败=0)
  进度: 30/32 (成功=30, 失败=0)
✅ PXRD计算完成: 全部32个成功
评估完成: 耗时 2.23秒

  最佳改进样本:
    A-967: 改进84.7% (CFG=1.50)
    A-809: 改进40.2% (CFG=1.50)
    A-49: 改进37.1% (CFG=1.50)

迭代2结果:
  满足要求: 0/200 (0.0%)
  平均RWP: 96.7025
  ✨ 本轮有样本得到改进

📝 submission.csv已更新 (迭代2)
   满足要求: 0/200 (0.0%)
   文件大小: 274.62 KB

--- 迭代 3 ---
已运行: 0.01小时
处理 32 个样本
  CFG策略：精确匹配，引导强度=2.25
  批量重新生成 32 个结构...
使用CFG采样: 2.25


## 11. 增量更新submission.csv函数

In [ ]:
# 文件I/O函数已移至inference_utils.py
# 这里直接使用导入的函数

## 14. 最终统计和验证

In [ ]:
# 最终统计（使用导入的函数）
print_final_statistics(sample_status, START_TIME, RWP_THRESHOLD, MAX_ATTEMPTS_PER_SAMPLE)

# 验证最终的提交文件
print(f"\n验证最终submission文件:")
if os.path.exists(SUBMISSION_FILE):
    # 重新读取文件以验证
    final_submission = pd.read_csv(SUBMISSION_FILE)
    print(f"  文件名: {SUBMISSION_FILE}")
    print(f"  文件大小: {os.path.getsize(SUBMISSION_FILE) / 1024:.2f} KB")
    print(f"  样本数: {len(final_submission)}")
    print(f"  列名: {final_submission.columns.tolist()}")
    
    # 检查是否有缺失值
    missing = final_submission.isnull().sum()
    if missing.any():
        print(f"\n⚠️ 警告：发现缺失值！")
        print(missing[missing > 0])
    else:
        print(f"  ✅ 没有缺失值")
    
    # 检查ID是否完整
    expected_ids = set(sample_status.keys())
    actual_ids = set(final_submission['ID'].values)
    if expected_ids == actual_ids:
        print(f"  ✅ 所有样本ID都已包含")
    else:
        missing_ids = expected_ids - actual_ids
        extra_ids = actual_ids - expected_ids
        if missing_ids:
            print(f"  ⚠️ 缺少ID: {missing_ids}")
        if extra_ids:
            print(f"  ⚠️ 多余ID: {extra_ids}")
else:
    print(f"  ❌ 文件不存在: {SUBMISSION_FILE}")

print("\n" + "="*60)
print("✅ 推理完成！submission.csv已在整个过程中实时更新")
print("="*60)


最终统计

质量统计:
  满足RWP<0.15: 0/200 (0.0%)
  未满足要求: 200

未满足要求样本的RWP:
  最小: 1.0548
  最大: 173.6884
  平均: 61.4070

尝试次数统计:
  最少: 3
  最多: 10
  平均: 9.5
  达到上限(10次): 169

总运行时间: 0.40小时

验证最终submission文件:
  文件名: submission.csv
  文件大小: 270.02 KB
  样本数: 200
  列名: ['ID', 'cif']
  ✅ 没有缺失值
  ✅ 所有样本ID都已包含

✅ 推理完成！submission.csv已在整个过程中实时更新


## 总结

本notebook实现了迭代优化的推理流程，并**实时更新submission.csv**：

### 核心流程
1. ✅ **初始推理**：模型推理生成初始结构，立即保存submission.csv
2. ✅ **质量评估**：使用RWP指标评估PXRD匹配质量
3. ✅ **迭代优化**：
   - 能量优化 + Rietveld精修
   - **每轮迭代后立即更新submission.csv**
   - 批量重新生成不满足要求的样本
4. ✅ **终止条件**：
   - 运行时间限制（5小时）
   - 单样本尝试次数限制
   - 全部满足要求

### 关键改进
- 🔄 **增量更新机制**：每次推理/优化后立即覆盖submission.csv
- 📊 **实时进度反馈**：评测脚本可以随时读取最新结果
- 🛡️ **断点续传支持**：即使中途中断，已有结果也保存在submission.csv中
- 📝 **更新日志**：每次更新都记录状态信息（满足率、文件大小等）

### 待实现部分
- ⏳ 实际的模型推理
- ⏳ PXRD计算（调用PXRDSimulator）
- ⏳ 能量优化（GULP等）
- ⏳ Rietveld精修（GSAS-II等）

### 输出
- ✅ 符合比赛要求的submission.csv（CIF格式）
- ✅ **实时更新**：每轮推理后立即保存，评测脚本可及时读取
- ✅ 详细的优化过程记录和统计